# Discussing Coroutines **Context**

In [1]:
// Importing some neccessary libs in jupyter notebook
@file:DependsOn("org.jetbrains.kotlinx:kotlinx-coroutines-core:1.5.2")
import kotlinx.coroutines.*
import java.util.concurrent.Executors

## Coroutine `Context`

- A coroutine **context** in Kotlin is a set of elements that define the behavior of a coroutine. It holds information such as the **dispatcher** (which determines which *thread* or *threads* the coroutine runs on), a **Job** that *controls the lifecycle* of the coroutine, and other optional elements that can customize coroutine execution.

1. `Job`
    - A `Job` controls the lifecycle of the coroutine. It can be used to track and manage the coroutine's state (active, canceled, or completed).
    - The `Job` is part of the **context**, and every coroutine has an associated `Job`.
3. `CoroutineDispatcher`
    - `Dispatchers.Default`: Uses a shared background pool of threads and is optimized for CPU-intensive tasks.
    - `Dispatchers.IO`: Optimized for I/O-bound tasks such as reading from files or network operations.
    - `Dispatchers.Main`: Confines the execution to the main thread, often used for updating UI components.
    - `Dispatchers.Unconfined`: The coroutine starts on the current thread but can resume on different threads. It is less commonly used.
    - **Customised Dispatcher**: It is possible to create a customised dispatcher with a given number of threads.
5. `CoroutineName`
   - A `CoroutineName` is an optional element that gives a name to a coroutine for debugging purposes.

In [2]:
// Launching a coroutine with a given context

// As we saw, 'runBlocking' can also accept a customised context like 'runBlocking(context) {}'
fun main() = runBlocking { 
    //.............runBlocking coroutine scope.................
    println("Program started \n `running on $coroutineContext`")

    // Let's create a customized context
    // The creations structure is like [CoroutineDispatcher]+[Job]+[CoroutineName]
    val context = Dispatchers.Default + CoroutineName("MyContext")

    // Then launch a coroutine using the created context
    val job = launch(context) {
        // 'coroutineContext' has the reference to the context of the coroutine
        println("Hello: \n `running on $coroutineContext`")
    }
    job.join()

    // Inherits its context from its outer scope, e.g. runBlocking
    async {
        println("world: \n  `running on $coroutineContext`")
    }.await() // We can also 'join' the returned job like this 
    
    println("Program finished \n `running on $coroutineContext`")
}

main()

Program started 
 `running on [BlockingCoroutine{Active}@44d17435, BlockingEventLoop@6f29e41d]`
Hello: 
 `running on [CoroutineName(MyContext), StandaloneCoroutine{Active}@dcb9eb2, Dispatchers.Default]`
world: 
  `running on [DeferredCoroutine{Active}@78c78a09, BlockingEventLoop@6f29e41d]`
Program finished 
 `running on [BlockingCoroutine{Active}@44d17435, BlockingEventLoop@6f29e41d]`


In [3]:
// Explicitly defining the 'Job' and launching a coroutine with a given context

fun main() = runBlocking { 
    //.............runBlocking coroutine scope.................
    println("Program started")

    // Let's create customized contexts
    
    /* 
    The creations structure is like [CoroutineDispatcher]+[Job]+[CoroutineName]
    'context1' inherits the 'Job' from the outer scope, e.g. 'runBlocking'. 
    */
    val context1 = Dispatchers.Default + CoroutineName("MyContext1")
    
    /* 
    'context2' has its own job and it can be used to control
    the lifecyle of the launched corotuines within 'context2'; there might be several coroutines
    launched in this context.
    */
    val contextJob = Job();
    val context2 = Dispatchers.Default + contextJob + CoroutineName("MyContext2")
    
    // Then launch a coroutine using the created context
    val job1 = launch(context1) {
        println("Hello: \n `running on $coroutineContext`")
    }
    job1.join()

    // Then launch a coroutine using the created context
    val job2 = async(context2) {
        println("World: \n `running on $coroutineContext`")
    }
    job2.await()

    /* 
    VERY INPORTANT: We are responsible for myScope lifecycle management
    because we have defined a dedicated 'job' for it;
    we must 'cancel()' its 'Job' when we are done.
    */
    context2.cancel()
    println("Program finished")
}

main()

Program started
Hello: 
 `running on [CoroutineName(MyContext1), StandaloneCoroutine{Active}@733f5c2, Dispatchers.Default]`
World: 
 `running on [CoroutineName(MyContext2), DeferredCoroutine{Active}@42317b40, Dispatchers.Default]`
Program finished


## Using `withContext` to dispatch coroutine execution

In [4]:
// Switching the context using 'withContext'

## Corouitne `Context` with `CoroutineScope`

In [5]:
// Assigning a customised context to a a custom coroutine scope

fun main() = runBlocking { 
    //.............runBlocking coroutine scope.................
    println("Program started: \n `running on $coroutineContext`")

    // Let's create customized contexts
    val contextJob = Job()
    val context = Dispatchers.IO + contextJob + CoroutineName("MyContext")

    // coroutineScope(context) {} // Will not work; this coroutine builder does not accept 'Context'
    
    // Inherits its context form the outer scope
    coroutineScope {
        println("Hello World 0: \n `running on $coroutineContext`")
        // There is an option to use 'launch' with the given context
        launch(context) { println("Hello World 1: \n `running on $coroutineContext`") }
    }

    // But we can directly define a scope and launch jobs in that scope
    val myScope = CoroutineScope(context)
    // Now, the launch notations is [My Coroutine Scope].[launch{}/asnyc{}]
    val job = myScope.launch { 
        println("Hello World 2: \n `running on $coroutineContext`")
    }
    job.join()
    /* 
    VERY INPORTANT: We are responsible for myScope lifecycle management
    because we have defined a dedicated 'job' for it;
    we must 'cancel()' its 'Job' when we are done.
    */
    myScope.cancel() 
    
    println("Program finished: \n `running on $coroutineContext`")
}

main()

Program started: 
 `running on [BlockingCoroutine{Active}@42898046, BlockingEventLoop@23154fdd]`
Hello World 0: 
 `running on [ScopeCoroutine{Active}@5d621cd5, BlockingEventLoop@23154fdd]`
Hello World 1: 
 `running on [CoroutineName(MyContext), StandaloneCoroutine{Active}@13de43ba, Dispatchers.IO]`
Hello World 2: 
 `running on [CoroutineName(MyContext), StandaloneCoroutine{Active}@54596e27, Dispatchers.IO]`
Program finished: 
 `running on [BlockingCoroutine{Active}@42898046, BlockingEventLoop@23154fdd]`


In [6]:
// Creating multi-threaded context

fun main() = runBlocking { 
    //.............runBlocking coroutine scope.................
    println("Program started: \n `running on $coroutineContext`")

    /* 
    VERY IMPORTANT: Threads are resource-intensive, so it's important to create them judiciously. 
    In most cases, it's better to rely on Kotlin's default dispatchers (Dispatchers.Default, Dispatchers.IO).
    */
    // Let's create customized multi-thread contexts
    val threadNo = 6
    val contextJob = Job()
    val contextDispatcher = Executors.newFixedThreadPool(threadNo).asCoroutineDispatcher()
    val context = contextDispatcher + contextJob + CoroutineName("MyContext")

    // But we can directly define a scope and launch jobs in that scope
    val myScope = CoroutineScope(context)
    
    // Launch multiple jobs in this custom scope
    val jobs = List(10) { i ->
        myScope.launch {
            println("Job $i started on thread: ${Thread.currentThread().name}")
            delay(1000L)  // Simulating some work
            println("Job $i finished on thread: ${Thread.currentThread().name}")
        }
    }

    // Wait for all jobs to complete
    jobs.joinAll()
    /* 
    VERY INPORTANT: We are responsible for myScope lifecycle management
    because we have defined a dedicated 'job' for it;
    we must 'cancel()' its 'Job' when we are done.
    */
    myScope.cancel() // cancels its job
    
    println("Program finished: \n `running on $coroutineContext`")
}

main()

Program started: 
 `running on [BlockingCoroutine{Active}@574ec766, BlockingEventLoop@778078e2]`
Job 1 started on thread: pool-1-thread-2
Job 2 started on thread: pool-1-thread-3
Job 0 started on thread: pool-1-thread-1
Job 3 started on thread: pool-1-thread-4
Job 4 started on thread: pool-1-thread-5
Job 5 started on thread: pool-1-thread-6
Job 6 started on thread: pool-1-thread-1
Job 7 started on thread: pool-1-thread-3
Job 8 started on thread: pool-1-thread-4
Job 9 started on thread: pool-1-thread-2
Job 2 finished on thread: pool-1-thread-6
Job 1 finished on thread: pool-1-thread-5
Job 0 finished on thread: pool-1-thread-1
Job 3 finished on thread: pool-1-thread-3
Job 4 finished on thread: pool-1-thread-4
Job 5 finished on thread: pool-1-thread-2
Job 7 finished on thread: pool-1-thread-5
Job 6 finished on thread: pool-1-thread-6
Job 8 finished on thread: pool-1-thread-1
Job 9 finished on thread: pool-1-thread-3
Program finished: 
 `running on [BlockingCoroutine{Active}@574ec766, Bloc